In [ ]:
# using TF

In [ ]:
....
....
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
            
        acc_train = accuracy.eval(feed_dict = {X: X_batch, y: y_batch})
        acc_test  = accuracy.eval(feed_dict = {X: mnist.test.images, y: mnist.test.labels})
        print(epoch, 'Train_accurayc:', acc_train, 'Test_accuracy:', acc_test)
    save_path = saver.save(sess, '/home/ramscrux7757/SPARK/MNIST/my_mode_final.ckpt')

In [ ]:
# Using the saved_chkpt
with tf.Session() as sess:
    saver.restore(sess, '/home/ramscrux7757/SPARK/MNIST/my_mode_final.ckpt')
    #X_test = [.....]
    Z = logits.eval(feed_dict={X: mnist.test.images})
    y_pred = np.argmax(Z, axis=1)
    print(y_pred)

In [ ]:
# using Keras

In [ ]:
# define the model
def define_model(vocab_size, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 100, input_length=max_length))
    model.add(Conv1D(filters=32, kernel_size=8, activation='relu')) 
    # kernel_size - is the number of words to consider as the convolutional is passed across the 
    # input text document
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(10,activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile the network
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    model.summary()
    return model

In [ ]:
model = define_model(vocab_size, max_length)
model.fit(Xtrain2, ytrain,epochs=50,verbose=1)

In [ ]:
model.save('/home/ramscrux7757/SPARK/SENT_ANALS/sent_model.h5')

In [ ]:
# Loading the model
from keras.models import load_model
model = load_model('/home/ramscrux7757/SPARK/SENT_ANALS/sent_model.h5')
# evaluate the model on the test data
_, acc = model.evaluate(Xtest2, ytest, verbose=0)
print('Test Accuracy: {}'.format(acc*100))

In [ ]:
# option 2: (this is better as it saves the better one !!!)
checkpoint = ModelCheckpoint(
    filepath=('/home/ramscrux7757/SPARK/AI_SCIENCE/qa-lstm-attn-best.hdf5'),
    verbose=1, save_best_only=True)

history = model.fit([Xqtrain, Xatrain], [Ytrain], batch_size=batch_size, 
                    nb_epoch=10, validation_split=0.1,callbacks=[checkpoint])

In [ ]:
# loading the model and evaluating the test data
from keras.models import load_model
svd_model = load_model('/home/ramscrux7757/SPARK/AI_SCIENCE/qa-lstm-attn-best.hdf5')
loss, acc = svd_model.evaluate([Xqtest, Xatest], [Ytest], batch_size=batch_size)
svd_model.predict([Xqtest, Xatest], batch_size=batch_size)

In [ ]:
# option 3:
# saving the model as separate files
from numpy import array
from keras.models import model_from_json

# convert the architecture to 'json' or 'yaml' file
architecture = model.to_json()
with open('/home/ramscrux7757/SPARK/AI_SCIENCE/architecture.json','wt') as json_file:
    json_file.write(architecture)

# saving the weights as *.hdf5 file
#model.save_weights('weights.h5')
#model.save_weights('/home/ramscrux7757/SPARK/AI_SCIENCE/weights.h5')

In [ ]:
# Loading the model
from keras.models import model_from_json
json_file = open('/home/ramscrux7757/SPARK/AI_SCIENCE/architecture.json','rt')
architecture = json_file.read()
json_file.close()
# create model from architecture
model = model_from_json(architecture)

# loading weights
model = model.load_weights('/home/ramscrux7757/SPARK/AI_SCIENCE/weights.h5')

# making predictions
yhat = model.predict(X, verbose=0)
yhat = model.predict_classes(X)
yhat = model.predict_proba(X)
print(yhat)